# Prediction with LSTM model (v.4.4.1)
<p style='text-align: right;'>with selectd4.csv</p>

* history
  * 2025/05/23 PM06:02 : 3번째 모델의 3번째 데이터를 사용

In [ ]:
!pip install scikit-learn
!pip install tensorflow
from datetime import datetime
print(f'restart kernel... {datetime.now()}')

In [1]:
def restart_kernel():
    # Restart the kernet after libraries are loaded.
    import IPython
    from datetime import datetime
    print(f'restart kernel... {datetime.now()}')
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

restart_kernel()

restart kernel... 2025-05-28 12:00:53.426582


In [1]:
# load dependacies
import os
import time
import resource
from datetime import datetime
from lib.globalvar import *
from lib.data_loader import DataLoader
from lib.small_predict import predict_small, print_predict_small
from lib.activation import ActivationOutput, RecurrentActivation
from lib.small_predict import build_and_predict, gen_multi_model, save_model, generate_metric
from lib.common_env_sets import lstm_units, dense_units, steps, metrics, dropout
from lib.common_env_sets import learning_rate, last_lstm_return_sequences
from lib.common_env_sets import loss, output_dense_activation, epochs, rand_seed

print(f"imported library. ({datetime.now()})")

imported library. (2025-05-28 13:17:27.631430)


In [4]:
##
## 모델링 환경 설정
##
window=1 # default = 1 , help = "time stamps"
data_dir='./csv/selectd4.csv'
    
mode='predict' # help = "back-test or predict")

mode2='sampling' # help = "greed or sampling")
verb='verbose' # help = "verbose or not_verb")
    
trial=20 # help = "how much trials to generate")
training_length=0.95 # default = 0.9)
epoch=20 # default = 3
batch=1 # default = 1
model_type='lstm4' # help = "lstm1 or lstm2")
hid_dim = 50
from_pos = 0
last = [[1, 5, 18, 20, 30, 35],
        [7, 9, 24, 40, 42, 44],
        [3, 6, 7, 11, 12, 17],
        [3, 13, 28, 34, 38, 42],
        [5, 12, 24, 29, 32, 42]]

print(f"finished to set environemnt. ({datetime.now()})")

finished to set environemnt. (2025-05-28 13:17:33.083592)


In [5]:
import tensorflow as tf
print(f'{tf.version}')

<module 'tensorflow._api.v2.version' from '/home/swhors/jupyter-workspace/finance/venv/lib/python3.11/site-packages/tensorflow/_api/v2/version/__init__.py'>


In [6]:
dataset_dicts = {
    1: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=0,
                  verbose=False
                  ),
    2: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=300,
                  verbose=False
                  )
    }
#print(f'test_X = {dataset_dicts[1].test_X}')
print(f'completed to load data. {datetime.now()}')

completed to load data. 2025-05-28 13:17:34.518477


In [7]:
###########
## layers
## LSTM Neural 계층 선언
###########
datasets = [dataset_dicts[1], # 0
            dataset_dicts[2], # 1
           ]

lstm_args = {1: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[0],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[0],
                          "dense_units": dense_units[0],
                          "output_dense_activation": output_dense_activation[0],
                          "loss": loss[0],
                          "rand_seed": rand_seed[0]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 1
                           },
                 "dataset": datasets[0]
                 },
             2: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[0],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[1],
                          "dense_units": dense_units[1],
                          "output_dense_activation": output_dense_activation[0],
                          "loss": loss[0],
                          "rand_seed": rand_seed[0]
                          },
                 "train": {"epochs": epochs[1],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 1
                           },
                 "dataset": datasets[0]
                 },
             3: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[0],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[1],
                          "dense_units": dense_units[1],
                          "output_dense_activation": output_dense_activation[0],
                          "loss": loss[0],
                          "rand_seed": rand_seed[1]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 1
                           },
                 "dataset": datasets[0]
                 },
             4: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[1],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[2],
                          "dense_units": dense_units[2],
                          "output_dense_activation": output_dense_activation[3],
                          "loss": loss[1],
                          "return_state": False,
                          "rand_seed": rand_seed[0] # rand_seed[0]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 3,
                           "steps_per_epoch": 90 #100
                           },
                 "dataset": datasets[0]
                 },
             5: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[1],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[3],
                          "dense_units": dense_units[3],
                          "output_dense_activation": output_dense_activation[3],
                          "loss": loss[1],
                          "rand_seed": rand_seed[0]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 3,
                           "steps_per_epoch": 100
                           },
                 "dataset": datasets[0]
                 },
             6: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[1],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[5],
                          "dense_units": dense_units[4],
                          "output_dense_activation": output_dense_activation[3],
                          "loss": loss[1],
                          "return_state": False,
                          "lstm_model": "Bidirectional",
                          "rand_seed": rand_seed[0] # rand_seed[0]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 3,
                           "steps_per_epoch": 90 #100
                           },
                 "dataset": datasets[0]
                 }
             }

print(f'completed to set evn for all models. {datetime.now()}')

completed to set evn for all models. 2025-05-28 13:17:35.716064


In [11]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
model_version_from = 16
model_version_end = 20
args = {'test_id': 'P4_4_1_1_1',
        'version': model_version_from,
        'write_to_db': True,
        'last': last,
        'mode2': mode2,
        'gen_num': 5,
        'round_limit': 40,
        'db_file_name': "./db/metrics.db"
       }
generate_metric(args=args, from_version=model_version_from, to_version=model_version_end, lstm_args=lstm_args)


********************
working model_version = 16  2025-05-28 13:45:30.786815
********************
start to train all models. 2025-05-28 13:45:30.787124
1's training. status=start 2025-05-28 13:45:30.787145
1's training. status=end 2025-05-28 13:46:29.418047
2's training. status=start 2025-05-28 13:46:29.418143
2's training. status=end 2025-05-28 13:46:49.439612
3's training. status=start 2025-05-28 13:46:49.439709
3's training. status=end 2025-05-28 13:47:34.962373
4's training. status=start 2025-05-28 13:47:34.962472
4's training. status=end 2025-05-28 13:48:16.882164
5's training. status=start 2025-05-28 13:48:16.882268
5's training. status=end 2025-05-28 13:48:52.400099
6's training. status=start 2025-05-28 13:48:52.400198
6's training. status=end 2025-05-28 13:49:11.085235
completed to train all models. 2025-05-28 13:49:11.085332
train to create model(16). 2025-05-28 13:49:11.085345
CPU 시간: 1684.304624 초
메모리 사용량: 2525104 KB
********************
working model_version = 17  2025-05-28

In [ ]:
view_model_info = False
if view_model_info:
    for i in range(MAX_MODEL_CNT):
        print(models[i][0].model.summary())

In [ ]:
# all : save model
for model in models:
    if model is not None:
        model[0].save()